# ENCODING 

# Imports

In [58]:
#%load_ext autoreload
#%autoreload 2
import os, sys

import sys
sys.path.append('/home/ubuntu/')
os.chdir('/home/ubuntu/')

In [59]:
import importlib
import src
importlib.reload(src)
import src
import src.encoding as enc
import shlex, subprocess
import pandas as pd
from pybedtools import BedTool
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt


In [60]:
datapath= "/mnt/data/"
resultspath = '/mnt/data/results/'

In [68]:
#should I add the copying of the genome file in my folder to the jupyter notebook?

#creation of the file containing the length of the human chromosomes
subprocess.run(f"mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e \
                        'select chrom, size from hg38.chromInfo' \
                        > {datapath}hg38.genome", shell = True)

# Sloping

#ORIGINAL ENCODE
#sloping the whole directory to a common  length of 400 nucleotides
#original ENCODE
path = f'{datapath}ENCODE/processed/'
directory1 = os.listdir(path)


for folder in directory1:
    #print(folder)
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        if 'peaks.crosslink.anno.filt.bed' in file :
            subprocess.run(f"slopBed -i {path}/{folder}/peaks.crosslink.anno.filt.bed \
                                    -g {datapath}hg38.genome -b 200 \
                                    > {path}/{folder}/peaks.crosslink.anno.filt.slop.bed", shell= True)



# Conversion to FASTA

#ORIGINAL ENCODE
#converting all of the eCLIP sequences to fasta 
#original ENCODE
path = f'{datapath}ENCODE/processed/'
directory1 = os.listdir(path)

for folder in directory1:
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        
        if 'peaks.crosslink.anno.filt.bed' in file :
            subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                    -bed {path}/{folder}/peaks.crosslink.anno.filt.slop.bed -fo \
                                    > {path}/{folder}/peaks.crosslink.anno.filt.slop.fa.out", shell= True)



#NEW NEGATIVES
# converting negatives into fasta
#New negative dataset

path = f'{datapath}negatives/'
directory1 = os.listdir(path)

for folder in directory1:
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                        -bed {path}{folder}/{file} -fo \
                                        > {path}{folder}/{file}.fa", shell= True)


# Removal of files (if necessary) 


In [ ]:
#removal of previous files 
#modified dataset

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:
    #if folder == 'RBM15_HepG2':
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if 'out.df' in file :
                subprocess.run(f"rm {path}{folder}/{folder2}/{file}", shell= True)



# preparing folders for plots 

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)

# tic()

for folder in directory1:
    if folder not in list_high_meth:
        current_folder = folder
        directory2 = os.listdir(f"{path}{folder}")
        subprocess.run(f"rmdir results/plots/summary_metrics/{folder}", shell= True)



#removal of previous files 
#modified dataset

path = f'{datapath}negatives/'
directory1 = os.listdir(path)


for folder in directory1:
    current_folder = folder
    directory3 = os.listdir(f"{path}{folder}")

    for file in directory3 :

        if '.miclip' in file :
            subprocess.run(f"rm {path}{folder}/{file}", shell= True)



# m6A sites filtering

#NEW NEGATIVES
#m6A sites filtering for new set of negatives 


path = f'{datapath}negatives/'
directory1 = os.listdir(path)

for folder in directory1: 
    current_folder = folder
    directory2 = os.listdir(path + '/' + folder)

    for file in directory2 :

        if '.bed.fa' not in file :
            subprocess.run(f"bedtools intersect -wb \
                                    -a {datapath}miCLIP.filt.out.bed \
                                    -b {path}{folder}/{file} \
                                     > {path}{folder}/{file}.miclip.filt.bed.out", shell= True)




#CV DATASET
#m6A sites filtering for pos, neg-1, neg-2 and folds --> for one hot encoding --> isn't the fasta missing? no, it's included in the cv dataset already


path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if '.bed' in file :
                subprocess.run(f"bedtools intersect -wb \
                                        -a {datapath}miCLIP.filt.out.bed \
                                        -b {path}/{folder}/{folder2}/{file} \
                                        > {path}/{folder}/{folder2}/{file}.miclip.filt.bed.out", shell= True)




# DATA AUGMENTATION

# just CV dataset

#first retrieving the RPB binding site from .bed
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:  
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :
            #this is just for positives, since negative 1 don't have to contain any RBP site  
            if ('.bed' in file ) and  ('out' not in file) :
                subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                    -g {datapath}hg38.genome -l -200 -r -200 \
                                    > {path}{folder}/{folder2}/{file}.out", shell= True)




#now I'm going to slop the coordinates randomly --> bed.out.slop.out

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:   
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :
        #considering the fact that negative 1 do not have to contain any RBP binding site
            
            if ('.bed.out' in file ) and  ('slop' not in file) and ('miclip' not in file):
                l,r = enc.random_l_r(399,2) #-l and -r should be randomized and their sum should be 400. 
                subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                        -g {datapath}hg38.genome -l {l} -r {r} \
                                        > {path}{folder}/{folder2}/{file}.slop.out", shell= True)

            

            if ('.bed.out' in file ) and ( 'negative' in file ) and  ('slop' not in file) and ('miclip' not in file):
                for i in [0, 1, 2]:
                    l,r = enc.random_l_r(399,2)            
                    subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                            -g {datapath}hg38.genome -l {l} -r {r} \
                                            > {path}{folder}/{folder2}/{file}.slop.out{i}", shell= True)
                    subprocess.run(f'cat {path}{folder}/{folder2}/{file}.slop.out{i} \
                                        >> {path}{folder}/{folder2}/{file}.slop.out', shell = True)  #here merge the files into {path}/{folder}/{folder2}/{file}.slop.out
                    subprocess.run(f'rm {path}{folder}/{folder2}/{file}.slop.out{i}', shell = True)
                    

#now I have the slopped sequences, I'll check if the m6A site is included -- the file is very similar to miclip.filt.bed.out , the code is same
#later to filter out out.slop.out.fasta file --> out.slop.out.miclip.filt.bed.out contains 

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if ('.out.slop.out' in file) and ('fasta' not in file) and ('miclip' not in file) :
                subprocess.run(f"bedtools intersect -wb \
                                        -a {datapath}miCLIP.filt.out.bed \
                                        -b {path}{folder}/{folder2}/{file} \
                                       > {path}{folder}/{folder2}/{file}.miclip.filt.bed.out", shell= True)




#now I have to convert the new sequences (out.slop.out)  into fasta

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if ('out.slop.out' in file) and ('.miclip.filt.bed' not in file) and ('fasta' not in file) :
                subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                            -bed {path}{folder}/{folder2}/{file} -fo {path}{folder}/{folder2}/{file}.fasta", shell= True)

                   


# New Augmentation

In [32]:

# define function to be used as lambda in the apply() later on
def get_center_pos_between_rbp_and_m6a(x0, x1):
    m6a_pos = x0 # this should be single-nucleotide so doesn't matter if you take start or end
    rbp_pos = x1
    dist = rbp_pos - m6a_pos # this is a positive quantity if m6a_pos left, negative otherwise
    return int( rbp_pos + dist / 2)



In [46]:
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)


for folder in directory1:
    if folder in list_high_meth:
    #if folder == 'RBM15_HepG2':
        current_folder = folder
        directory2 = os.listdir(f"{path}{folder}")

        for folder2 in directory2 :
            directory3 = os.listdir(f"{path}{folder}/{folder2}")

            for file in directory3 :
                if ('miclip.filt.bed.out' in file) and ('df' not in file): 
                    try: 
                        bed = BedTool(f'{path}{folder}/{folder2}/{file}')
                        df = bed.to_dataframe(disable_auto_names=True, names = ["chrom_m6a", "start_m6a", "end_m6a", "details",'score', 'strand',"chrom_gene","start_gene","end_gene","ids", 'thickStart', 'thickEnd' ,'a', "chrom_rbp", "start_rbp", "end_rbp", 'b', 'c','d'])
                        #df = pd.DataFrame(f'{path}{folder}/{folder2}/{file}', sep="\t", names=["chrom_m6a", "start_m6a", "end_m6a", "details","chrom_gene","start_gene","end_gene", "ids" ,"chrom_rbp", "start_rbp", "end_rbp"])
                        # get original bindingsite position
                        df['bs_pos'] = df['start_rbp'].apply(lambda x: x + 200)
                        df['pos_between_rbp_and_m6a'] = df[['bs_pos', 'start_m6a']].apply(lambda x: enc.get_center_pos_between_rbp_and_m6a(x[0], x[1]), axis=1)
                        df['chrom'] = df['chrom_m6a']
                        df['end_pos_between_rbp_and_m6a']= df['pos_between_rbp_and_m6a']
                        cols = df.columns.tolist()
                        cols = ['chrom','pos_between_rbp_and_m6a','end_pos_between_rbp_and_m6a',"chrom_m6a", "start_m6a", "end_m6a", "details",'score', 'strand',"chrom_gene","start_gene","end_gene","ids", 'thickStart', 'thickEnd' ,'a', "chrom_rbp", "start_rbp", "end_rbp", 'b', 'c','d','bs_pos']
                        df = df[cols]
                        #print(df)
                        BedTool.from_dataframe(df,f'{path}{folder}/{folder2}/{file}.df' )
                    except: 
                        print(f'error occurred in {path}{folder}/{folder2}/{file}')
                        continue

positive.fold-4.bed.miclip.filt.bed.out
     chrom  pos_between_rbp_and_m6a  end_pos_between_rbp_and_m6a chrom_m6a   
0    chr10                110590432                    110590432     chr10  \
1    chr10                110590430                    110590430     chr10   
2    chr10                110601766                    110601766     chr10   
3    chr10                110601732                    110601732     chr10   
4    chr10                 17229514                     17229514     chr10   
..     ...                      ...                          ...       ...   
732   chrX                135033052                    135033052      chrX   
733   chrX                135033043                    135033043      chrX   
734   chrX                135033064                    135033064      chrX   
735   chrX                 44524182                     44524182      chrX   
736   chrX                 72272593                     72272593      chrX   

     start_m6a    end_m

In [53]:
#now I'm going to slop the coordinates randomly --> bed.out.slop.out
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)


for folder in directory1:
    if folder in list_high_meth: 
    #if folder == 'RBM15_HepG2':
        current_folder = folder
        directory2 = os.listdir(f"{path}{folder}")

        for folder2 in directory2 :
            directory3 = os.listdir(f"{path}{folder}/{folder2}")

            for file in directory3 :
                #considering the fact that negative 1 do not have to contain any RBP binding site

                if ('miclip.filt.bed.out.df' in file ) and ('positive' in file) and ('slop' not in file):
                    l,r = enc.random_l_r(400,2) #-l and -r should be randomized and their sum should be 400. 
                    subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                            -g {datapath}hg38.genome -l {l} -r {r} \
                                            > {path}{folder}/{folder2}/{file}.slop", shell= True)



                if ('miclip.filt.bed.out.df' in file ) and ('negative' in file) and ('slop' not in file):
                    for i in range(30):
                        l,r = enc.random_l_r(400,2)            
                        subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                                -g {datapath}hg38.genome -l {l} -r {r} \
                                                > {path}{folder}/{folder2}/{file}.slop{i}", shell= True)
                        subprocess.run(f'cat {path}{folder}/{folder2}/{file}.slop{i} \
                                            >> {path}{folder}/{folder2}/{file}.slop', shell = True)  #here merge the files into {path}/{folder}/{folder2}/{file}.slop.out
                        subprocess.run(f'rm {path}{folder}/{folder2}/{file}.slop{i}', shell = True)



In [48]:
#now I have to convert the new sequences (out.slop.out)  into fasta
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)


for folder in directory1:
    if folder in list_high_meth: 
    #if folder == 'RBM15_HepG2':
        current_folder = folder
        directory2 = os.listdir(f"{path}{folder}")

        for folder2 in directory2 :
            directory3 = os.listdir(f"{path}{folder}/{folder2}")

            for file in directory3 :

                if ('miclip.filt.bed.out.df.slop' in file) and ('fasta' not in file) :
                    subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                                -bed {path}{folder}/{folder2}/{file} -fo {path}{folder}/{folder2}/{file}.fasta", shell= True)




In [54]:
#now I have the slopped sequences, I'll check if the m6A site is included -- the file is very similar to miclip.filt.bed.out , the code is same
#later to filter out out.slop.out.fasta file --> out.slop.out.miclip.filt.bed.out contains 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)


for folder in directory1:
    if folder in list_high_meth: 
    #if folder == 'RBM15_HepG2': 
        current_folder = folder
        directory2 = os.listdir(f"{path}{folder}")

        for folder2 in directory2 :
            directory3 = os.listdir(f"{path}{folder}/{folder2}")

            for file in directory3 :

                if ('miclip.filt.bed.out.df.slop' in file) and ('fasta' not in file) :
                    subprocess.run(f"bedtools intersect -wb \
                                            -a {datapath}miCLIP.filt.out.bed \
                                            -b {path}{folder}/{folder2}/{file} \
                                           > {path}{folder}/{folder2}/{file}.miclip.filt.bed.out", shell= True)

